# Парсинг банковских рейтингов с сайта banki.ru / Parsing bank ratings from site banki.ru

In [1]:
import requests
import re
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd


### Так выглядит  главная ссылка, в которой мы будем изменять номер страницы / It's main reference, in which  we will change number of page: 

In [32]:
main_url = "http://www.banki.ru/banks/ratings/agency/?PAGEN_2={}&mode=2"

In [33]:
page = 1

In [34]:
requests.get(main_url.format(page))

<Response [200]>

200 ответ --> соединение установлено и данные получены, всё чудесно! **/** 200 answer --> the connection is set and the data is got, good! 

Создадим переменную, которая будет держать HTML код нашей полученной страницы **/** Create var, which will keep HTML-code of got page.

In [35]:
ratings_page = requests.get(main_url.format(page))
ratings_page = ratings_page.content

In [ ]:
ratings_page

Немного структуируем весь этот ужас / Structure this horror :

In [37]:
ratings_page = BeautifulSoup(ratings_page, "lxml")

In [ ]:
ratings_page

## Зная блок  элемента на странице, мы можем находить его в этой куче html-кода / Knowing the block of element on the page, we can find it in this heap of html-code.

**Извлечём номера лицензий и соответсвующие им названия банков / Extract license numbers and corresponding them bank names.**

In [39]:
print(ratings_page.find('div', attrs = {'class':"bank-info"}))

<div class="bank-info">No.2306, Москва и обл.</div>


In [40]:
for num_lic in ratings_page.findAll('div', attrs = {'class':"bank-info"}):
    print(num_lic)

<div class="bank-info">No.2306, Москва и обл.</div>
<div class="bank-info">No.2879, Москва и обл.</div>
<div class="bank-info">No.415, Татарстан</div>
<div class="bank-info">No.1455, Татарстан</div>
<div class="bank-info">No.2880, Москва и обл.</div>
<div class="bank-info">No.2860, Саратовская обл.</div>
<div class="bank-info">No.1810, Амурская обл.</div>
<div class="bank-info">No.2590, Татарстан</div>
<div class="bank-info">No.2587, Татарстан</div>
<div class="bank-info">No.2529, Мордовия</div>
<div class="bank-info">No.3006, Самарская обл.</div>
<div class="bank-info">No.567, Новосибирская обл.</div>
<div class="bank-info">No.53, Санкт-Петербург и обл.</div>
<div class="bank-info">No.2119, Москва и обл.</div>
<div class="bank-info">No.2602, Саха (Якутия)</div>
<div class="bank-info">No.2388, Алтайский край</div>
<div class="bank-info">No.2659, Алтайский край</div>
<div class="bank-info">No.1326, Москва и обл.</div>
<div class="bank-info">No.608, Москва и обл.</div>
<div class="bank-i

In [41]:
def html_stripper(text):
    return re.sub('<[^<]+?>', '', str(text)) # очень крутые регулярные выражения - убирают все скобочки, тэги и прочую ерунду

In [42]:
cnt = 0
for cont in ratings_page.findAll('div', attrs = {'class':"b-bank-name"}):
    print(cnt)
    print(re.split('No.|,', html_stripper(cont))[:2])
    print("-------------------------")
    cnt+=1

0
['Абсолют Банк', '2306']
-------------------------
1
['Авангард', '2879']
-------------------------
2
['Аверс', '415']
-------------------------
3
['Автоградбанк', '1455']
-------------------------
4
['Агропромкредит', '2880']
-------------------------
5
['Агророс', '2860']
-------------------------
6
['Азиатско-Тихоокеанский Банк', '1810']
-------------------------
7
['Ак Барс', '2590']
-------------------------
8
['Акибанк', '2587']
-------------------------
9
['Актив Банк', '2529']
-------------------------
10
['АктивКапитал Банк', '3006']
-------------------------
11
['Акцепт', '567']
-------------------------
12
['Александровский', '53']
-------------------------
13
['Алеф-Банк', '2119']
-------------------------
14
['Алмазэргиэнбанк', '2602']
-------------------------
15
['АлтайБизнес-Банк', '2388']
-------------------------
16
['Алтайкапиталбанк', '2659']
-------------------------
17
['Альфа-Банк', '1326']
-------------------------
18
['Аспект', '608']
------------------------

In [43]:
num_lics = []
bank_names = []

for cont in ratings_page.findAll('div', attrs = {'class':"b-bank-name"}):
    name, num = re.split('No.|,', html_stripper(cont))[:2]
    num_lics += [num]
    bank_names += [name]

num_lics = list(map(int,num_lics))

In [44]:
num_lics

[2306,
 2879,
 415,
 1455,
 2880,
 2860,
 1810,
 2590,
 2587,
 2529,
 3006,
 567,
 53,
 2119,
 2602,
 2388,
 2659,
 1326,
 608,
 732,
 2929,
 101,
 2748,
 1067,
 128]

In [45]:
bank_names

['Абсолют Банк',
 'Авангард',
 'Аверс',
 'Автоградбанк',
 'Агропромкредит',
 'Агророс',
 'Азиатско-Тихоокеанский Банк',
 'Ак Барс',
 'Акибанк',
 'Актив Банк',
 'АктивКапитал Банк',
 'Акцепт',
 'Александровский',
 'Алеф-Банк',
 'Алмазэргиэнбанк',
 'АлтайБизнес-Банк',
 'Алтайкапиталбанк',
 'Альфа-Банк',
 'Аспект',
 'Ассоциация',
 'ББР Банк',
 'БКС — Инвестиционный Банк',
 'БМ-Банк (бывш. Банк Москвы)',
 'БайкалИнвестБанк',
 'Балтийский Банк']

In [48]:
def bank_names_and_lics(ratings_page):
    num_lics = []
    bank_names = []

    for cont in ratings_page.findAll('div', attrs = {'class':"b-bank-name"}):
        name, num = re.split('No.|,', html_stripper(cont))[:2]
        num_lics += [num]
        bank_names += [name]

    num_lics = list(map(int,num_lics))
    
    return (num_lics, bank_names)

In [ ]:
cnt=0
for cell in ratings_page.findAll('div', attrs = {"class":"b-table-ratings"}):
    cnt+=1
    print(cnt)
    #print(html_stripper(cell))
    print(cell)
    print("-------------------------------")

In [ ]:
ratings_page.findAll('div', attrs = {"class":"b-table-ratings"})

In [51]:
sp=re.split('<td>|</td>', str(ratings_page.findAll('div', attrs = {"class":"b-table-ratings"})).replace('\n', ' ').replace("<!--","").replace("-->","").replace('[',"").replace(']',""))

In [52]:
sp = list(map(html_stripper, sp))

In [53]:
sp = [el for el in sp if len(el.split())!= 0]

In [54]:
len(sp)

205

In [ ]:
for el in sp:
    print(el.split()[:4])

In [56]:
ratings  = pd.DataFrame()

In [57]:
s_MOODYS = ["P-1","P-2","P-3","NP","отозван"]

on_nat_MOODYS = ["Aaa.ru","Aa1.ru","Aa2.ru","Aa3.ru","A1.ru","A2.ru","A3.ru",\
                "Baa1.ru","Baa2.ru","Baa3.ru","Ba1.ru","Ba2.ru","Ba3.ru","B1.ru",\
                 "B2.ru","B3.ru","Caa1.ru","Caa2.ru","Caa3.ru","Ca.ru","C.ru",\
                    "отозван"]

fin_stab_MOODYS = ["A","B+","B","B-","C+","C","C-","D+","D","D-","E+","E","E-","отозван"]


In [ ]:
flag = 1
new_line = None
bank_ind = 0
num_lics = banks_name_and_lic(ratings_page)[0] 
bank_names = banks_name_and_lic(ratings_page)[1]
for i in range(len(sp)):
    l = sp[i].split()[:4]
    print(i,l)
    
    add_to_df = 0
    if flag:
        new_line = dict()
        flag=0
        
    if "Moody's" in l:
        if "Рус-рейтинг" in sp[i+1].split()[:4]:
            new_line["l_in_foreign_MOODYS"] = np.nan
            new_line["l_in_nat_MOODYS"] = np.nan
            new_line["MOODYS"] = np.nan
            new_line["s_in_foreign_MOODYS"] = np.nan
            new_line["s_in_nat_MOODYS"] = np.nan
            new_line["on_nat_MOODYS"] = np.nan
            new_line["fin_stab_MOODYS"] = np.nan
            new_line["dates_MOODYS"] = np.nan
        
        else:
            new_line["l_in_foreign_MOODYS"] = l[1]
            new_line["l_in_nat_MOODYS"] = l[2]
            new_line["MOODYS"] = l[3]
            
            if sp[i+1].split()[:2][0] in s_MOODYS:
                new_line["s_in_foreign_MOODYS"] = sp[i+1].split()[:4][0]
            
            if sp[i+1].split()[:2][1] in s_MOODYS:
                new_line["s_in_nat_MOODYS"] = sp[i+1].split()[:4][1]
            
            if sp[i+2].split()[:4][0] in on_nat_MOODYS:
                if len(sp[i+3].split()[:4][0]) == 10:
                    new_line["on_nat_MOODYS"] = np.nan
                else:
                    new_line["on_nat_MOODYS"] = sp[i+2].split()[:4][0]
                    
            if sp[i+2].split()[:4][0] in fin_stab_MOODYS: 
                new_line["fin_stab_MOODYS"] = sp[i+2].split()[:4][0]
                
            elif len(sp[i+2].split()[:4][0]) == 10:
                new_line["dates_MOODYS"] = sp[i+2].split()[:4][0]
                
            if sp[i+3].split()[:4][0] in fin_stab_MOODYS: 
                new_line["fin_stab_MOODYS"] = sp[i+3].split()[:4][0]
                
            elif len(sp[i+3].split()[:4][0]) == 10:
                new_line["dates_MOODYS"] = sp[i+3].split()[:4][0]
            
            if len(sp[i+4].split()[:4][0]) == 10:
                new_line["dates_MOODYS"] = sp[i+4].split()[:4][0]
            
            
    if "Эксперт" in l:
        if "НРА" in sp[i+1].split()[:4]:
            new_line["EXPERT_RA"] = np.nan
            new_line["EXPERT_RA_prognosis"] = np.nan
            new_line["EXPERT_RA_dates"] = np.nan
        else:
            new_line["EXPERT_RA"] = sp[i+1].split()[:4][0]
            if new_line["EXPERT_RA"] == sp[i+1].split()[:4][1]:
                new_line["EXPERT_RA_prognosis"] = np.nan
            else:
                new_line["EXPERT_RA_prognosis"] = sp[i+1].split()[:4][1]
            
            new_line["EXPERT_RA_dates"] = sp[i+2].split()[:4][0]
            
    if "НРА" in l:
        if 'AK&M' in sp[i+1].split()[:4]:
            new_line["НРА"] = np.nan
            new_line["НРА_prognosis"] = np.nan
            new_line["НРА_dates"] = np.nan
        else:
            new_line["НРА"] = sp[i+1].split()[:4][0]
            if new_line["НРА"] == sp[i+1].split()[:4][1]:
                new_line["НРА_prognosis"]= np.nan
            else:
                new_line["НРА_prognosis"] = sp[i+1].split()[:4][1]
                
            new_line["НРА_dates"] = sp[i+2].split()[:4][0]
            
    if "AK&M" in l:
        try:
            if "Moody's" in sp[i+1].split()[:4]:
                new_line["AK&M"] = np.nan
                new_line["AK&M_prognosis"] = np.nan
                new_line["AK&M_dates"] = np.nan
            
            else:
                new_line["AK&M"] = sp[i+1].split()[:4][0]
                if new_line["AK&M"] == sp[i+1].split()[:4][1]:
                    new_line["AK&M_prognosis"]= np.nan
                else:
                    new_line["AK&M_prognosis"] = sp[i+1].split()[:4][1]
                
                new_line["AK&M_dates"] = sp[i+2].split()[:4][0]
        except:
            new_line["AK&M"] = np.nan
            new_line["AK&M_prognosis"] = np.nan
            new_line["AK&M_dates"] = np.nan    
        
        flag = 1
        
    if flag:
        new_line["num_of_lic"] = num_lics[bank_ind]
        new_line["bank_name"] = bank_names[bank_ind]

        bank_ind += 1
        ratings = ratings.append(new_line, ignore_index=True)
        flag = 0


In [60]:
ratings = ratings[["num_of_lic","bank_name","l_in_foreign_MOODYS","l_in_nat_MOODYS", "MOODYS", "s_in_foreign_MOODYS", "s_in_nat_MOODYS", "on_nat_MOODYS", "fin_stab_MOODYS", "dates_MOODYS", "EXPERT_RA", "EXPERT_RA_prognosis","EXPERT_RA_dates","НРА","НРА_prognosis", "НРА_dates","AK&M","AK&M_prognosis","AK&M_dates"]]
ratings.head()

,num_of_lic,bank_name,l_in_foreign_MOODYS,l_in_nat_MOODYS,MOODYS,s_in_foreign_MOODYS,s_in_nat_MOODYS,on_nat_MOODYS,fin_stab_MOODYS,dates_MOODYS,EXPERT_RA,EXPERT_RA_prognosis,EXPERT_RA_dates,НРА,НРА_prognosis,НРА_dates,AK&M,AK&M_prognosis,AK&M_dates
0,2306.0,Абсолют Банк,B1,B1,негативный,NP,NP,отозван,E+,26.04.2016,ruBBB,стабильный,28.04.2017,NaN,NaN,NaN,NaN,NaN,NaN
1,2879.0,Авангард,B2,B2,стабильный,NP,NP,отозван,E+,18.03.2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,415.0,Аверс,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ruA+,стабильный,28.04.2017,NaN,NaN,NaN,NaN,NaN,NaN
3,1455.0,Автоградбанк,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ruBB-,стабильный,27.04.2017,NaN,NaN,NaN,NaN,NaN,NaN
4,2880.0,Агропромкредит,B2,B2,негативный,NP,NP,NaN,E+,21.04.2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Вишенка на тортике. Функция, делающая всё:) / The func, which does all necessary.

In [62]:
def bankiru_parsing(pages_number: int, verbose = False):
    """
    return: DataFrame table corresponding banki.ru table
    """
    main_url = "http://www.banki.ru/banks/ratings/agency/?PAGEN_2={}&mode=2"
    
    ratings = pd.DataFrame()
    
    #unique moodys values, just lists of constants 
    s_MOODYS = ["P-1","P-2","P-3","NP","отозван"]

    on_nat_MOODYS = ["Aaa.ru","Aa1.ru","Aa2.ru","Aa3.ru","A1.ru","A2.ru","A3.ru",\
                "Baa1.ru","Baa2.ru","Baa3.ru","Ba1.ru","Ba2.ru","Ba3.ru","B1.ru",\
                 "B2.ru","B3.ru","Caa1.ru","Caa2.ru","Caa3.ru","Ca.ru","C.ru",\
                    "отозван"]

    fin_stab_MOODYS = ["A","B+","B","B-","C+","C","C-","D+","D","D-","E+","E","E-","отозван"]

    for page in range(1, pages_number+1):
        
        if verbose:
            print(page)
        ratings_page = requests.get(main_url.format(page))
        ratings_page = ratings_page.content
        ratings_page = BeautifulSoup(ratings_page, "lxml")
    
        sp=re.split('<td>|</td>', str(ratings_page.findAll('div', attrs = {"class":"b-table-ratings"})).replace('\n', ' ').replace("<!--","").replace("-->","").replace('[',"").replace(']',""))
        sp = list(map(html_stripper, sp))
        sp = [el for el in sp if len(el.split())!= 0]
    
        
        flag = 1
        new_line = None
        bank_ind = 0
        num_lics = bank_names_and_lics(ratings_page)[0] 
        bank_names = bank_names_and_lics(ratings_page)[1]
        for i in range(len(sp)):    
            l = sp[i].split()[:4]
            if flag:
                new_line = dict()
                flag=0
        
            if "Moody's" in l:
                if "Рус-рейтинг" in sp[i+1].split()[:4]:
                    new_line["l_in_foreign_MOODYS"] = np.nan
                    new_line["l_in_nat_MOODYS"] = np.nan
                    new_line["MOODYS"] = np.nan
                    new_line["s_in_foreign_MOODYS"] = np.nan
                    new_line["s_in_nat_MOODYS"] = np.nan
                    new_line["on_nat_MOODYS"] = np.nan
                    new_line["fin_stab_MOODYS"] = np.nan
                    new_line["dates_MOODYS"] = np.nan
                elif len(l) == 1: #special code for 2312 num lic
                    new_line["l_in_foreign_MOODYS"] = np.nan
                    new_line["l_in_nat_MOODYS"] = np.nan
                    new_line["MOODYS"] = np.nan
                    new_line["s_in_foreign_MOODYS"] = np.nan
                    new_line["s_in_nat_MOODYS"] = np.nan
                    if sp[i+1].split()[:4][0] in on_nat_MOODYS:
                        if len(sp[i+1].split()[:4][0]) == 10:
                            new_line["on_nat_MOODYS"] = np.nan
                        else:
                            new_line["on_nat_MOODYS"] = sp[i+1].split()[:4][0]
                    
                    new_line["fin_stab_MOODYS"] = np.nan

                    if len(sp[i+2].split()[:4][0]) == 10:
                        new_line["dates_MOODYS"] = sp[i+2].split()[:4][0]
                else:
                    
                    new_line["l_in_foreign_MOODYS"] = l[1]
                    new_line["l_in_nat_MOODYS"] = l[2]
                    new_line["MOODYS"] = l[3]

                    if sp[i+1].split()[:2][0] in s_MOODYS:
                        new_line["s_in_foreign_MOODYS"] = sp[i+1].split()[:4][0]
            
                    if sp[i+1].split()[:2][1] in s_MOODYS:
                        new_line["s_in_nat_MOODYS"] = sp[i+1].split()[:4][1]
            
                    if sp[i+2].split()[:4][0] in on_nat_MOODYS:
                        if len(sp[i+3].split()[:4][0]) == 10:
                            new_line["on_nat_MOODYS"] = np.nan
                        else:
                            new_line["on_nat_MOODYS"] = sp[i+2].split()[:4][0]
                    
                    if sp[i+2].split()[:4][0] in fin_stab_MOODYS: 
                        new_line["fin_stab_MOODYS"] = sp[i+2].split()[:4][0]
                
                    elif len(sp[i+2].split()[:4][0]) == 10:
                        new_line["dates_MOODYS"] = sp[i+2].split()[:4][0]
                
                    if sp[i+3].split()[:4][0] in fin_stab_MOODYS: 
                        new_line["fin_stab_MOODYS"] = sp[i+3].split()[:4][0]
                
                    elif len(sp[i+3].split()[:4][0]) == 10:
                        new_line["dates_MOODYS"] = sp[i+3].split()[:4][0]
            
                    if len(sp[i+4].split()[:4][0]) == 10:
                        new_line["dates_MOODYS"] = sp[i+4].split()[:4][0]
                        
            if "Эксперт" in l:
                if "НРА" in sp[i+1].split()[:4]:
                    new_line["EXPERT_RA"] = np.nan
                    new_line["EXPERT_RA_prognosis"] = np.nan
                    new_line["EXPERT_RA_dates"] = np.nan
                else:
                    new_line["EXPERT_RA"] = sp[i+1].split()[:4][0]
                    if new_line["EXPERT_RA"] == sp[i+1].split()[:4][1]:
                        new_line["EXPERT_RA_prognosis"] = np.nan
                    else:
                        new_line["EXPERT_RA_prognosis"] = sp[i+1].split()[:4][1]
            
                    new_line["EXPERT_RA_dates"] = sp[i+2].split()[:4][0]
            
            if "НРА" in l:
                if 'AK&M' in sp[i+1].split()[:4]:
                    new_line["НРА"] = np.nan
                    new_line["НРА_prognosis"] = np.nan
                    new_line["НРА_dates"] = np.nan
                else:
                    new_line["НРА"] = sp[i+1].split()[:4][0]
                    if new_line["НРА"] == sp[i+1].split()[:4][1]:
                        new_line["НРА_prognosis"]= np.nan
                    else:
                        new_line["НРА_prognosis"] = sp[i+1].split()[:4][1]
                
                    new_line["НРА_dates"] = sp[i+2].split()[:4][0]
            
            if "AK&M" in l:
                try:
                    if "Moody's" in sp[i+1].split()[:4]:
                        new_line["AK&M"] = np.nan
                        new_line["AK&M_prognosis"] = np.nan
                        new_line["AK&M_dates"] = np.nan
            
                    else:
                        new_line["AK&M"] = sp[i+1].split()[:4][0]
                        if new_line["AK&M"] == sp[i+1].split()[:4][1]:
                            new_line["AK&M_prognosis"]= np.nan
                        else:
                            new_line["AK&M_prognosis"] = sp[i+1].split()[:4][1]
                
                        new_line["AK&M_dates"] = sp[i+2].split()[:4][0]
                except:
                    new_line["AK&M"] = np.nan
                    new_line["AK&M_prognosis"] = np.nan
                    new_line["AK&M_dates"] = np.nan    
        
                flag = 1
        
            if flag:
                new_line["num_of_lic"] = num_lics[bank_ind]
                new_line["bank_name"] = bank_names[bank_ind]
                ratings = ratings.append(new_line, ignore_index=True)
                bank_ind += 1
                flag = 0 

    ratings = ratings[["num_of_lic", "bank_name","l_in_foreign_MOODYS","l_in_nat_MOODYS", "MOODYS", "s_in_foreign_MOODYS", "s_in_nat_MOODYS", "on_nat_MOODYS", "fin_stab_MOODYS", "dates_MOODYS", "EXPERT_RA", "EXPERT_RA_prognosis","EXPERT_RA_dates","НРА","НРА_prognosis", "НРА_dates","AK&M","AK&M_prognosis","AK&M_dates"]]

    return ratings

### На сайте banki.ru есть 11 страниц таблицы с рейтингами о банках / In banki.ru there are 11 table pages with ratings of banks.

In [63]:

data=bankiru_parsing(pages_number = 11)

In [64]:
data

,num_of_lic,bank_name,l_in_foreign_MOODYS,l_in_nat_MOODYS,MOODYS,s_in_foreign_MOODYS,s_in_nat_MOODYS,on_nat_MOODYS,fin_stab_MOODYS,dates_MOODYS,EXPERT_RA,EXPERT_RA_prognosis,EXPERT_RA_dates,НРА,НРА_prognosis,НРА_dates,AK&M,AK&M_prognosis,AK&M_dates
0,2306.0,Абсолют Банк,B1,B1,негативный,NP,NP,отозван,E+,26.04.2016,ruBBB,стабильный,28.04.2017,NaN,NaN,NaN,NaN,NaN,NaN
1,2879.0,Авангард,B2,B2,стабильный,NP,NP,отозван,E+,18.03.2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,415.0,Аверс,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ruA+,стабильный,28.04.2017,NaN,NaN,NaN,NaN,NaN,NaN
3,1455.0,Автоградбанк,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ruBB-,стабильный,27.04.2017,NaN,NaN,NaN,NaN,NaN,NaN
4,2880.0,Агропромкредит,B2,B2,негативный,NP,NP,NaN,E+,21.04.2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2860.0,Агророс,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ruBB,негативный,14.08.2017,NaN,NaN,NaN,NaN,NaN,NaN
6,1810.0,Азиатско-Тихоокеанский Банк,отозван,отозван,отозван,отозван,отозван,отозван,отозван,11.05.2016,ruCC,негативный,19.04.2017,NaN,NaN,NaN,NaN,NaN,NaN
7,2590.0,Ак Барс,B2,B2,стабильный,NP,NP,отозван,E+,12.05.2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2587.0,Акибанк,B3,B3,стабильный,NP,NP,отозван,E+,18.03.2016,ruBB+,стабильный,26.06.2017,NaN,NaN,NaN,NaN,NaN,NaN
9,2529.0,Актив Банк,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ruBBB-,стабильный,22.06.2017,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
data.to_csv("bank_ratings.csv",index=False)